<a href="https://colab.research.google.com/github/muke888/DataScience-Competitions/blob/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/Richter_Prediction_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost -q
!pip install scikit-optimize -q
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold

from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper,  DeltaXStopper, CheckpointSaver #,VerboseCallback
from skopt.space import Real, Categorical, Integer

from sklearn.metrics import roc_auc_score
import copy

X = pd.read_csv("https://raw.githubusercontent.com/muke888/DataScience-Competitions/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/train_values.csv")
y = pd.read_csv("https://raw.githubusercontent.com/muke888/DataScience-Competitions/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/train_labels.csv")
test_value = pd.read_csv("https://raw.githubusercontent.com/muke888/DataScience-Competitions/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/test_values.csv")
#submission_format = pd.read_csv("https://raw.githubusercontent.com/muke888/DataScience-Competitions/master/002.Richter's%20Predictor%3A%20Modeling%20Earthquake%20Damage/submission_format.csv")

     |████████████████████████████████| 66.1MB 43kB/s 
     |████████████████████████████████| 81kB 4.1MB/s 


In [ ]:
submission_format.head()

In [ ]:
X.head()

In [ ]:
X.nunique()

In [ ]:
#No NaN values, thank god!
X = X.astype(str)
X[['count_floors_pre_eq', 'age', 'area_percentage','height_percentage','count_families']] = X[['count_floors_pre_eq', 'age', 'area_percentage','height_percentage','count_families']].astype(np.int64) 
#X[[ 'age', 'area_percentage','height_percentage']] = X[['age', 'area_percentage','height_percentage']].astype(np.int32) 
X.info()

categorical_features_indices = np.where(X.dtypes != np.int64)[0]
print(categorical_features_indices)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  object
 1   geo_level_1_id                          260601 non-null  object
 2   geo_level_2_id                          260601 non-null  object
 3   geo_level_3_id                          260601 non-null  object
 4   count_floors_pre_eq                     260601 non-null  int64 
 5   age                                     260601 non-null  int64 
 6   area_percentage                         260601 non-null  int64 
 7   height_percentage                       260601 non-null  int64 
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 

In [ ]:
y.info()
y.head()
#print (y.groupby('damage_grade')['building_id'].nunique(),'\n', y_train.value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
#Need to stratify to ensure equal proportions of label classes
X_train, X_validation, y_train, y_validation = train_test_split(X, y.damage_grade, train_size=0.8, random_state=888, stratify=y.damage_grade)

X_test = test_value

In [ ]:
'''model = CatBoostClassifier(
    loss_function = 'MultiClass', #'MultiClassOneVsAll'
    eval_metric = 'Accuracy',
    custom_loss=['Accuracy', 'TotalF1'],
    #random_seed=42,
    iterations=10000,
    #learning_rate=0.03,
    #logging_level='Silent',
    #border_count=32,)'''

params = {
    'loss_function': 'MultiClass', #'MultiClassOneVsAll'
    'eval_metric': 'TotalF1',  
    #'custom_loss':['Accuracy', 'TotalF1'],
    'iterations': 19512,  #13327  
    'task_type': 'CPU',
    'use_best_model': True,
    #'logging_level':'Silent',
    #'random_seed': 888,
    'bagging_temperature': 0.5406467228757142, #0.9554779863099283,
    'border_count':118, #better with default 254?
    'depth': 8,
    'l2_leaf_reg': 16, #13
    'learning_rate': 0.05151153261112777,# 0.02647454924455449
    'random_strength': 4.40226865948004e-07# 1.0014067340536191e-09
    #'ignored_features':[0],
    #'logging_level': 'Silent',
    #'boosting_type':'Ordered', #only 'Plain' works for multiclass
}

train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

model = CatBoostClassifier(**params)

In [ ]:
model.fit(train_pool,
    eval_set= validate_pool,
    #logging_level='Verbose',  # you can uncomment this for text output
    plot=False,
    verbose=100
);

In [ ]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [ ]:
cv_params = model.get_params()
#cv_params.update({
#    'loss_function': 'Logloss'
#})
cv_data = cv(
    Pool(X, y.damage_grade, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

In [ ]:
pred=model.predict(X_test)
pred

In [ ]:
d = {'building_id': X_test.building_id.values,'damage_grade': pred.flatten() }
df=pd.DataFrame(d)
df

In [ ]:
df.to_csv('out_bayes_01.csv', index=False)
#cv_data.to_csv('cv_data2.csv', index=False)

In [ ]:
model.get_all_params()


In [ ]:
#https://scikit-optimize.github.io/stable/auto_examples/sklearn-gridsearchcv-replacement.html#sphx-glr-auto-examples-sklearn-gridsearchcv-replacement-py
#https://www.kaggle.com/lucamassaron/kaggle-days-paris-skopt-catboost-solution

In [ ]:
import time
from skopt.callbacks import CheckpointSaver
start_time = time.time()

clf = CatBoostClassifier(#thread_count=2,
cat_features = categorical_features_indices,
loss_function = 'MultiClass',
eval_metric = 'TotalF1',
task_type= 'GPU',
random_seed=888,
early_stopping_rounds=300,
iterations=10000,
verbose = 1000)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

search_spaces = {'iterations': Integer(5000, 20000),
                 'depth': Integer(7, 15),
                 'learning_rate': Real(0.01, 0.05, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 #'ctr_border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30)}
search_spaces2 = {#'iterations': Integer(1000, 3000),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30),
                 #'scale_pos_weight':Real(0.01, 1.0, 'uniform')
                  }
# Setting up BayesSearchCV
opt = BayesSearchCV(clf,
                    search_spaces2,
                    #scoring=roc_auc,
                    cv=skf,
                    n_iter=50,#def=50
                    n_jobs=1,  # use just 1 job with CatBoost in order to avoid segmentation fault
                    #return_train_score=False,
                    refit=True,
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=88)

# callback handler
def on_step(optim_result):
    score = opt.best_score_
    print("best score: %s" % score)
    print("--- %s minutes ---" % round((time.time() - start_time)/60))
    if score >= 0.98:
        print('Interrupting!')
        return True

checkpoint_saver = CheckpointSaver("./checkpoint.pkl", compress=9)

opt.fit(X, y.damage_grade, callback=[on_step, checkpoint_saver] )#, [DeadlineStopper(60*1),DeltaXStopper(0.1)])

0:	learn: 0.4125940	total: 16.9ms	remaining: 2m 48s
1000:	learn: 0.7044314	total: 12.6s	remaining: 1m 52s
2000:	learn: 0.7072014	total: 25.1s	remaining: 1m 40s
3000:	learn: 0.7089534	total: 37.6s	remaining: 1m 27s
4000:	learn: 0.7096122	total: 50.1s	remaining: 1m 15s
5000:	learn: 0.7102767	total: 1m 2s	remaining: 1m 2s
6000:	learn: 0.7106087	total: 1m 15s	remaining: 50s
7000:	learn: 0.7109543	total: 1m 27s	remaining: 37.5s
8000:	learn: 0.7112354	total: 1m 39s	remaining: 25s
9000:	learn: 0.7112874	total: 1m 52s	remaining: 12.5s
9999:	learn: 0.7114181	total: 2m 5s	remaining: 0us
0:	learn: 0.4125994	total: 14.3ms	remaining: 2m 22s
1000:	learn: 0.7055775	total: 12.6s	remaining: 1m 52s
2000:	learn: 0.7084400	total: 25.1s	remaining: 1m 40s
3000:	learn: 0.7098337	total: 37.7s	remaining: 1m 27s
4000:	learn: 0.7108428	total: 50.3s	remaining: 1m 15s
5000:	learn: 0.7112250	total: 1m 2s	remaining: 1m 2s
6000:	learn: 0.7115084	total: 1m 15s	remaining: 50.1s
7000:	learn: 0.7120639	total: 1m 27s	rema

KeyboardInterrupt: ignored

In [ ]:
print("val. score: %s" % opt.best_score_)
opt.best_params_


val. score: 0.7442220098925176


OrderedDict([('bagging_temperature', 0.9925390711064718),
             ('border_count', 207),
             ('depth', 8),
             ('l2_leaf_reg', 2),
             ('learning_rate', 0.013368790818723922),
             ('random_strength', 0.00012737613815826226)])

In [ ]:
#train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
#validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

def objective(trial):
    X_train_c = copy.deepcopy(X_train)
    y_train_c = copy.deepcopy(y_train)
    X_validation_c = copy.deepcopy(X_validation)
    y_validation_c = copy.deepcopy(y_validation)
    categorical_features_indices_c = copy.deepcopy(categorical_features_indices)
    train_pool = Pool(X_train_c, y_train_c, cat_features=categorical_features_indices_c)
    validate_pool = Pool(X_validation_c, y_validation_c, cat_features=categorical_features_indices_c)


    param = {

        'loss_function': 'MultiClass',
        'eval_metric': 'Accuracy',
        'task_type': 'GPU',
        'use_best_model': True,
        #'logging_level':'Silent',
        'verbose' : 3500,
        #'random_seed': 888,
        'iterations': 20000, #trial.suggest_int('iterations', 10000, 20000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.06),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-9, 10),
        'depth': trial.suggest_int('depth', 6, 16),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 2, 30),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0.0, 1.0)
    }

    gbm = CatBoostClassifier(**param)

    gbm.fit(train_pool,
    eval_set= validate_pool,
    #logging_level='Verbose',  # you can uncomment this for text output
    plot=False,
    early_stopping_rounds=5000
    #verbose=1
    )

    preds = gbm.predict(X_validation_c)
    #pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_validation_c, preds)

    del gbm, train_pool, validate_pool, X_train_c, y_train_c, X_validation_c, y_validation_c, preds, categorical_features_indices_c

    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, gc_after_trial=True)

In [ ]:
study.trials_dataframe()